In [1]:
%%bash
export version=`python --version |awk '{print $2}' |awk -F"." '{print $1$2}'`

echo $version

if [ $version == '36' ] || [ $version == '37' ]; then
    echo 'Starting installation...'
    pip3 install pyspark==2.4.8 wget==3.2 pyspark2pmml==0.5.1 > install.log 2> install.log
    if [ $? == 0 ]; then
        echo 'Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)'
    else
        echo 'Installation failed, please check log:'
        cat install.log
    fi
elif [ $version == '38' ] || [ $version == '39' ]; then
    pip3 install pyspark==3.1.2 wget==3.2 pyspark2pmml==0.5.1 > install.log 2> install.log
    if [ $? == 0 ]; then
        echo 'Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)'
    else
        echo 'Installation failed, please check log:'
        cat install.log
    fi
else
    echo 'Currently only python 3.6, 3.7 , 3.8 and 3.9 are supported, in case you need a different version please open an issue at https://github.com/IBM/claimed/issues'
    exit -1
fi

37
Starting installation...
Please <<RESTART YOUR KERNEL>> (Kernel->Restart Kernel and Clear All Outputs)


In [14]:
from pyspark import SparkContext, SparkConf, SQLContext
import os
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark2pmml import PMMLBuilder
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import MinMaxScaler
import logging
import shutil
import site
import sys
import wget
import re

In [16]:
if sys.version[0:3] == '3.9':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.7.2/'
           'jpmml-sparkml-executable-1.7.2.jar')
    wget.download(url)
    shutil.copy('jpmml-sparkml-executable-1.7.2.jar',
                site.getsitepackages()[0] + '/pyspark/jars/')
elif sys.version[0:3] == '3.8':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.7.2/'
           'jpmml-sparkml-executable-1.7.2.jar')
    wget.download(url)
    shutil.copy('jpmml-sparkml-executable-1.7.2.jar',
                site.getsitepackages()[0] + '/pyspark/jars/')
elif sys.version[0:3] == '3.7':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.5.12/'
           'jpmml-sparkml-executable-1.5.12.jar')
    wget.download(url)
elif sys.version[0:3] == '3.6':
    url = ('https://github.com/jpmml/jpmml-sparkml/releases/download/1.5.12/'
           'jpmml-sparkml-executable-1.5.12.jar')
    wget.download(url)
else:
    raise Exception('Currently only python 3.6 , 3.7, 3,8 and 3.9 is supported, in case '
                    'you need a different version please open an issue at '
                    'https://github.com/IBM/claimed/issues')

In [17]:
trends_csv = os.environ.get('trends_csv',
                              'trends.csv')  # input file name (csv)
master = os.environ.get('master',
                        "local[*]")  # URL to Spark master
model_target = os.environ.get('model_target',
                              "model.xml")  # model output file name
data_dir = os.environ.get('data_dir',
                          'data/')  # temporary directory for data. 
input_columns = os.environ.get('input_columns',
                               '["x", "y", "z"]')  # input columns to consider

In [18]:
parameters = list(
    map(lambda s: re.sub('$', '"', s),
        map(
            lambda s: s.replace('=', '="'),
            filter(
                lambda s: s.find('=') > -1 and bool(re.match(r'[A-Za-z0-9_]*=[.\/A-Za-z0-9]*', s)),
                sys.argv
            )
    )))

for parameter in parameters:
    logging.warning('Parameter: ' + parameter)
    exec(parameter)

In [19]:
conf = SparkConf().setMaster(master)
#if sys.version[0:3] == '3.6' or sys.version[0:3] == '3.7':
conf.set("spark.jars", 'jpmml-sparkml-executable-1.5.12.jar')

sc = SparkContext.getOrCreate(conf)
sqlContext = SQLContext(sc)
spark = sqlContext.sparkSession

In [46]:
#df = spark.read.csv(data_dir + trends_csv)
df = spark.read.option("header", True).csv(data_dir + trends_csv)

In [47]:
df

DataFrame[x: string, y: string, z: string, source: string, class: string]

In [22]:
# register a corresponding query table
df.createOrReplaceTempView('df')

In [48]:
from pyspark.sql.types import DoubleType
df = df.withColumn("x", df.x.cast(DoubleType()))
df = df.withColumn("y", df.y.cast(DoubleType()))
df = df.withColumn("z", df.z.cast(DoubleType()))

In [49]:
#df.drop()
df.show()

+----+----+----+--------------------+--------+
|   x|   y|   z|              source|   class|
+----+----+----+--------------------+--------+
|33.0|36.0|51.0|Accelerometer-201...|Eat_meat|
|33.0|36.0|51.0|Accelerometer-201...|Eat_meat|
|33.0|35.0|53.0|Accelerometer-201...|Eat_meat|
|31.0|37.0|52.0|Accelerometer-201...|Eat_meat|
|32.0|36.0|52.0|Accelerometer-201...|Eat_meat|
|32.0|36.0|51.0|Accelerometer-201...|Eat_meat|
|32.0|36.0|51.0|Accelerometer-201...|Eat_meat|
|33.0|36.0|53.0|Accelerometer-201...|Eat_meat|
|33.0|35.0|52.0|Accelerometer-201...|Eat_meat|
|33.0|36.0|52.0|Accelerometer-201...|Eat_meat|
|32.0|35.0|53.0|Accelerometer-201...|Eat_meat|
|33.0|36.0|52.0|Accelerometer-201...|Eat_meat|
|32.0|38.0|53.0|Accelerometer-201...|Eat_meat|
|32.0|37.0|52.0|Accelerometer-201...|Eat_meat|
|33.0|35.0|52.0|Accelerometer-201...|Eat_meat|
|32.0|36.0|53.0|Accelerometer-201...|Eat_meat|
|32.0|36.0|53.0|Accelerometer-201...|Eat_meat|
|32.0|36.0|52.0|Accelerometer-201...|Eat_meat|
|34.0|36.0|52

In [50]:
splits = df.randomSplit([0.8, 0.2], 1)
df_train = splits[0]
df_test = splits[1]

In [51]:
indexer = StringIndexer(inputCol="class", outputCol="label")

vectorAssembler = VectorAssembler(inputCols=eval(input_columns),
                                  outputCol="features")

normalizer = MinMaxScaler(inputCol="features", outputCol="features_norm")

In [53]:
# lr = LogisticRegression(maxIter=1000, regParam=0.01, elasticNetParam=1.0)
TA = [10, 20]
DA = [5, 7]

for i in TA:
    for j in DA:
        rfr = RandomForestClassifier(numTrees=i, maxDepth=j, seed=1)
        print("\nNum of Trees=" + str(i) + " maxDepth=" + str(j) + "\n")
        pipeline = Pipeline(stages=[indexer, vectorAssembler, normalizer, rfr])
        model = pipeline.fit(df_train)

        prediction = model.transform(df_train)

        binEval = MulticlassClassificationEvaluator(). \
            setMetricName("accuracy"). \
            setPredictionCol("prediction"). \
            setLabelCol("label")

        print(binEval.evaluate(prediction)) 
        
        #model = rfr.fit(df_train)
        #prediction = model.transform(df_train)
        
        #binEval = MulticlassClassificationEvaluator(). \
        #    setMetricName("accuracy"). \
        #    setPredictionCol("prediction"). \
        #    setLabelCol("label")

        #print(binEval.evaluate(prediction)) 
        


Num of Trees=10 maxDepth=5



0.43897645510792577

Num of Trees=10 maxDepth=7



0.4632436518379574

Num of Trees=20 maxDepth=5



0.44374702539264815

Num of Trees=20 maxDepth=7



[Stage 120:==========================================>              (6 + 2) / 8]

0.4671827318793919
